In [ ]:
# import simulation stuff
import conf_solo12_jump_step_adjustment_full_kinematics_mpc as conf
from robot_properties_solo.solo12wrapper import Solo12Config
from robot_properties_solo.solo12wrapper import Solo12Robot
from bullet_utils.env import BulletEnvWithGround
from simulate_solo import Simulator

# initialize robot in pybullet
env = BulletEnvWithGround()
simulator = Simulator(env, Solo12Robot(), conf)
robot = Solo12Config.buildRobotWrapper()
q0, dq0 = simulator.q0, simulator.nu0
simulator.robot.reset_state(q0, dq0) 
simulator.robot.pin_robot.framesForwardKinematics(q0)

step_length = conf.gait['stepLength']
w = 0.5*conf.step_adjustment_bound
for i, c in enumerate(conf.contact_sequence):
    for debri in c:
        if debri.ACTIVE:
            p = debri.pose.translation
            simulator.build_one_stepstone(
                start_pos=(p[0]-w, p[1], p[2]-0.017), orientation=(0., 0., 0, 1),
                stone_length=conf.step_adjustment_bound, stone_width=conf.step_adjustment_bound   
    )

In [ ]:
# import models and solvers
from centroidal_plus_double_integrator_kinematics_acados_solver import CentroidalPlusLegKinematicsAcadosSolver
from centroidal_plus_double_integrator_kinematics_casadi_model import CentroidalPlusLegKinematicsCasadiModel
from wholebody_croccodyl_solver import WholeBodyDDPSolver
from wholebody_croccodyl_model import WholeBodyModel
import numpy as np

# DDP warm-start
wbd_model = WholeBodyModel(conf)
ddp_planner = WholeBodyDDPSolver(wbd_model, MPC=False, WARM_START=False)
ddp_planner.solve()
ddp_sol = ddp_planner.get_solution_trajectories()
centroidal_warmstart = ddp_sol['centroidal']
q_warmstart = ddp_sol['jointPos']
qdot_warmstart = ddp_sol['jointVel']
x_warmstart = []
u_warmstart = []
rmodel, rdata = conf.rmodel, conf.rdata
for k in range(len(centroidal_warmstart)):
    x_warmstart.append(
        np.concatenate(
            [centroidal_warmstart[k],
             q_warmstart[k], 
             qdot_warmstart[k]]
            )
        )
    u_warmstart.append(np.concatenate([np.zeros(30)]))
# nominal traj-opt
model_nom = CentroidalPlusLegKinematicsCasadiModel(conf, STOCHASTIC_OCP=False)
solver_nom = CentroidalPlusLegKinematicsAcadosSolver(
    model_nom, x_warmstart, u_warmstart, MPC=True
)
x_nom, u_nom, lqr_gains = solver_nom.solve()

In [ ]:
import matplotlib.pylab as plt
import pinocchio as pin
import numpy as np
import utils

# visualize motion in meshcat
dt = conf.dt
dt_ctrl = conf.dt_ctrl
N_ctrl =  int(dt/dt_ctrl)
# load robot in meshcat viewer
if conf.WITH_MESHCAT_DISPLAY:
    viz = pin.visualize.MeshcatVisualizer(
    robot.model, robot.collision_model, robot.visual_model)
    try:
        viz.initViewer(open=True)
    except ImportError as err:
        print(err)
        sys.exit(0)
    viz.loadViewerModel()
# add nominal contact surfaces
s = conf.step_adjustment_bound
for i, contacts in enumerate(conf.contact_sequence):
    for contact_idx, contact in enumerate(contacts):
        if contact.ACTIVE:
            t = contact.pose.translation
            if contact.CONTACT == 'FR' or contact.CONTACT == 'FL':
                utils.addViewerBox(
                    viz, 'world/contact'+str(i)+str(contact_idx), 
                    s, s, 0., [1., .2, .2, .5]
                    )
            if contact.CONTACT == 'HR' or contact.CONTACT == 'HL':
                utils.addViewerBox(
                    viz, 'world/contact'+str(i)+str(contact_idx),
                     s, s, 0., [.2, .2, 1., .5]
                    )       
            utils.applyViewerConfiguration(
                viz, 'world/contact'+str(i)+str(contact_idx), 
                [t[0], t[1], t[2]-0.017, 1, 0, 0, 0]
                )

# visualize DDP warm-start 
for traj_idx in range(conf.N-1):
    q_warmstart_curr = q_warmstart[traj_idx]
    for ctrl_idx in range(N_ctrl): 
        viz.display(q_warmstart_curr)
    

# visualize SQP solution
# kinematic functions
ee_fk = model_nom.casadi_model.ee_fk
ee_jacobians = model_nom.casadi_model.ee_jacobians
f_total = []
q  = []
qdot = []
qddot = []
for traj_idx in range(conf.N-1):
    q_base = np.array(
            model_nom.casadi_model.q_plus(
                x_warmstart[traj_idx][3:7], x_nom[traj_idx, 12:15]*dt
                )
            ).squeeze()
    q_base_next = np.array(
            model_nom.casadi_model.q_plus(
                x_warmstart[traj_idx+1][3:7], x_nom[traj_idx+1, 12:15]*dt
                )
            ).squeeze() 
    # get generalized positions
    q_des = np.concatenate(
        [
            x_nom[traj_idx, 9:12], 
            q_base,
            x_nom[traj_idx, 15:27]
        ]
    )
    q_des_next = np.concatenate(
        [
            x_nom[traj_idx+1, 9:12], 
            q_base_next,
            x_nom[traj_idx+1, 15:27]]
        )
    # get generalized velocities
    v_des = x_nom[traj_idx, 27:]
    v_des_next = x_nom[traj_idx+1, 27:]
    
    # get generalized velocities
    a_des = u_nom[traj_idx, 12:]
    a_des_next = u_nom[traj_idx+1, 12:]

    # interpolate solution for low-level torque control
    q_interpol, qdot_interpol, qddot_interpol, f_interpol = utils.interpolate_one_step(
        rmodel, dt, dt_ctrl, 
        q_des, q_des_next, 
        v_des, v_des_next,
        a_des, a_des_next,
        u_nom[traj_idx, :12],  u_nom[traj_idx+1, :12]
    )
    for ctrl_idx in range(N_ctrl): 
        f_total += [f_interpol[ctrl_idx]]
        q += [q_interpol[ctrl_idx]]
        qdot += [qdot_interpol[ctrl_idx]]
        qddot += [qddot_interpol[ctrl_idx]]
        viz.display(q_interpol[ctrl_idx, :])
# plot optimized contact forces
f_total = np.array(f_total) 
q = np.array(q)
qdot = np.array(qdot)
qddot = np.array(qddot)
time = np.arange(0, np.round((f_total.shape[0])*dt_ctrl, 2), dt_ctrl)
for contact in conf.ee_frame_names:
    plt.rc('font', family ='serif')
    contact_name = contact[0:2]
    if contact_name == 'FL':
      f_scp_nom = f_total[:, 0:3]
    elif contact_name == 'FR':
      f_scp_nom = f_total[:, 3:6] 
    elif contact_name == 'HL':
      f_scp_nom = f_total[:, 6:9] 
    elif contact_name == 'HR':
      f_scp_nom = f_total[:, 9:12]
    fig, (fx, fy, fz) = plt.subplots(3, 1, sharex=True) 
    fx.step(time, f_scp_nom[:, 0], label=' contact forces nom (N)')
    fx.legend()
    fx.set_title('force x')
    fy.step(time, f_scp_nom[:, 1])
    fy.set_title('force y')
    fz.step(time, f_scp_nom[:, 2])
    fz.set_title('force z')
    fz.set_xlabel('Time (s)', fontsize=14)
    fig.suptitle('contact forces of '+ str(contact[0:2]))

# plot joint positions,velocities and accelerations
time = np.arange(0, np.round((q.shape[0])*dt_ctrl, 2), dt_ctrl)
for contact in conf.ee_frame_names:
    plt.rc('font', family ='serif')
    contact_name = contact[0:2]
    if contact_name == 'FL':
        q_plot = q[:, 7:10]
        qdot_plot = qdot[:, 6:9]
        qddot_plot = qddot[:, 6:9]
    elif contact_name == 'FR':
        q_plot = q[:, 10:13]
        qdot_plot = qdot[:, 9:12]
        qddot_plot = qddot[:, 9:12]
    elif contact_name == 'HL':
        q_plot = q[:, 13:16]
        qdot_plot = qdot[:, 12:15]
        qddot_plot = qddot[:, 12:15]
    elif contact_name == 'HR':
        q_plot = q[:, 16:19]
        qddot_plot = qdot[:, 15:18]
        qddot_plot = qddot[:, 15:18]
    # joint positions
    fig1, (HAA, HFE, KFE) = plt.subplots(3, 1, sharex=True)
    fig1.suptitle(str(contact[0:2])+ ' joint positions (rad)')
    HAA.set_title('HAA')
    HAA.step(time, q_plot[:, 0])
    HFE.set_title('HFE')
    HFE.step(time, q_plot[:, 1])
    KFE.set_title('KFE')
    KFE.step(time, q_plot[:, 2])
    KFE.set_xlabel('Time (s)', fontsize=14)    
    # joint velocities
    fig2, (HAA, HFE, KFE) = plt.subplots(3, 1, sharex=True)
    fig2.suptitle(str(contact[0:2])+ ' joint velocities (rad/s)')
    HAA.set_title('HAA')
    HAA.step(time, qdot_plot[:, 0])
    HFE.set_title('HFE')
    HFE.step(time, qdot_plot[:, 1])
    KFE.set_title('KFE')
    KFE.step(time, qdot_plot[:, 2])
    KFE.set_xlabel('Time (s)', fontsize=14)   
    # joint accelerations
    fig3, (HAA, HFE, KFE) = plt.subplots(3, 1, sharex=True)
    fig3.suptitle(str(contact[0:2])+ ' joint accelerations (rad/s^2)')
    HAA.set_title('HAA')
    HAA.step(time, qddot_plot[:, 0])
    HFE.set_title('HFE')
    HFE.step(time, qddot_plot[:, 1])
    KFE.set_title('KFE')
    KFE.step(time, qddot_plot[:, 2])
                  

In [ ]:
from centroidal_plus_double_integrator_kinematics_acados_solver import CentroidalPlusLegKinematicsAcadosSolver
from centroidal_plus_double_integrator_kinematics_casadi_model import CentroidalPlusLegKinematicsCasadiModel
import conf_solo12_jump_step_adjustment_full_kinematics_mpc as conf
from wholebody_croccodyl_solver import WholeBodyDDPSolver
from wholebody_croccodyl_model import WholeBodyModel
from utils import interpolate_one_step
import pinocchio as pin
import numpy as np
import pybullet
import time

# get timings
dt = conf.dt
dt_ctrl = conf.dt_ctrl
N_ctrl =  int(dt/dt_ctrl)
# joint PD gains
Kp = 2.0*np.eye(12)
Kd = 0.2*np.eye(12)
# save torques
tau_feedfwd = []
tau_feedbck = []
tau_total = []
# get end-effector jacobian functions
ee_jacobians = model_nom.casadi_model.ee_jacobians
pybullet.configureDebugVisualizer(pybullet.COV_ENABLE_GUI,0)
if model_nom._STOCHASTIC_OCP:
    logger = pybullet.startStateLogging(pybullet.STATE_LOGGING_VIDEO_MP4, 'jump_mpc_hike_stoch.mp4')
else:
    logger = pybullet.startStateLogging(pybullet.STATE_LOGGING_VIDEO_MP4, 'jump_mpc_hike_nom.mp4')
nx, nu = solver_nom.nx, solver_nom.nu
# trajectory references
N_traj, N_mpc = solver_nom.N_traj, solver_nom.N_mpc
# get acados solver object
solver = solver_nom.acados_solver
# create open-loop tuples
solver_nom.X_sim = np.zeros((N_traj, solver_nom.N_mpc+1, nx))
solver_nom.U_sim = np.zeros((N_traj, solver_nom.N_mpc, nu))
# create closed-loop tuples
X_sol = np.zeros((N_traj, nx))
U_sol = np.zeros((N_traj, nu))
x0 = np.concatenate([solver_nom.x_init[0][:9],
                     solver_nom.x_init[0][9:12],
                     np.zeros(3),
                     solver_nom.x_init[0][16:28],
                     solver_nom.x_init[0][28:31], 
                     np.zeros(3), 
                     solver_nom.x_init[0][34:]])
model = model_nom
# initial warm-start 
x_warm_start_N = np.copy(solver_nom.x_ref_mpc[:N_mpc])
u_warm_start_N = np.copy(solver_nom.u_ref_mpc[:N_mpc])
# trajector reference
x_ref = solver_nom.x_init
# sinulation loop
for traj_time_idx in range(N_traj):
    # update ocp
    solver_nom.update_ocp(traj_time_idx, x_warm_start_N, u_warm_start_N, x0)  
    # solve OCP
    if solver_nom.ocp.solver_options.nlp_solver_type == 'SQP_RTI':
        # QP preparation rti_phase:
#         print('starting RTI preparation phase ' + '...')
        solver.options_set('rti_phase', 1)
        t_prep = time.time()
        status = solver.solve()
        elapsed_prep = time.time() - t_prep
#         print('RTI preparation phase took ' + str(elapsed_prep) + " seconds")
        # feedback rti_phase
#         print('starting RTI feedback phase ' + '...')
        solver.options_set('rti_phase', 2)
        t_feedback = time.time()
        status = solver.solve()
        elapsed_feedback = time.time() - t_feedback
#         print('RTI feedback phase took ' + str(elapsed_feedback) + " seconds")
        solver.print_statistics()
#         if status == 0:
#             print("HOORAY ! found a solution after :", 
#             elapsed_prep+elapsed_feedback, " seconds")
#         else:
#             raise Exception(f'acados returned status {status}.')
    else:
        t = time.time()
        status = solver.solve()
        elapsed_time= time.time() - t
        solver.print_statistics()
        if status == 0:
            print("HOORAY ! found a solution after :", 
            elapsed_time, " seconds")
        else:
            raise Exception(f'acados returned status {status}.')
    # get solution
    x_sol = np.array([solver.get(i,"x") for i in range(N_mpc+1)])
    u_sol = np.array([solver.get(i,"u") for i in range(N_mpc)])
    # get desired generalized position
    q_des = np.concatenate(
        [
            x_sol[0, 9:12],
            np.array(
                model_nom.casadi_model.q_plus(
                    x_warmstart[traj_time_idx][12:16], x_sol[0, 12:15]*dt
                    )
                ).squeeze(),
            x_sol[0, 15:27]
        ]
    )
    q_des_next = np.concatenate(
        [
            x_sol[1, 9:12],
            np.array(
                model_nom.casadi_model.q_plus(
                    x_warmstart[traj_time_idx+1][12:16], x_sol[1, 12:15]*dt
                    )
                ).squeeze(),
            x_sol[1, 15:27]
        ]
    )        
    # get desired generalized velocity
    v_des = x_sol[0, 27:]
    v_des_next = x_sol[1, 27:]
    # get desired generalized acceleration
    a_des = u_sol[0, 12:]
    a_des_next = u_sol[1, 12:]
    # get desired contact forces
    f_des = u_sol[0, :12]
    f_des_next = u_sol[1, :12]
    # interpolate solution for low-level torque control
    q_interpol, qdot_interpol, qddot_interpol, f_interpol = interpolate_one_step(
        rmodel, dt, dt_ctrl,
        q_des, q_des_next,
        v_des, v_des_next,
        a_des, a_des_next,
        f_des, f_des_next
    )
    # torque control loop
    for ctrl_time_idx in range(N_ctrl):
        q_interpol_k = q_interpol[ctrl_time_idx]
        q_meas, qdot_meas = simulator.robot.get_state()
        # compute J.T @ f
        Jtf = ee_jacobians[1](q=q_interpol_k)['J'][:3, :].T @ (f_interpol[ctrl_time_idx, 0:3]) +\
              ee_jacobians[0](q=q_interpol_k)['J'][:3, :].T @ (f_interpol[ctrl_time_idx, 3:6]) +\
              ee_jacobians[3](q=q_interpol_k)['J'][:3, :].T @ (f_interpol[ctrl_time_idx, 6:9]) +\
              ee_jacobians[2](q=q_interpol_k)['J'][:3, :].T @ (f_interpol[ctrl_time_idx, 9:12])
        # compute inverse dynamics
        tau_ff = np.array(
            model_nom.casadi_model.rnea(
                q_interpol_k, qdot_interpol[ctrl_time_idx], qddot_interpol[ctrl_time_idx]
            )
        )[6:].squeeze() - np.array(Jtf)[6:].squeeze()
        q_meas, qdot_meas = simulator.robot.get_state()
        tau_fb = Kp @ (q_interpol[ctrl_time_idx, 7:] - q_meas[7:]) + \
                 Kd @ (qdot_interpol[ctrl_time_idx, 6:] - qdot_meas[6:])
        # tau = t_ff + K_p @ (q_d - q_meas) + K_d @ (qdot_d - qdot_meas)
        tau_k = tau_ff + tau_fb
        simulator.robot.send_joint_command(tau_k)
        # step simulation
        simulator.env.step(sleep=False)        
    # warm-start solver from the previous solution 
    x_warm_start_N = np.concatenate([x_sol[1:], x_sol[-1].reshape(1, nx)]) #x_sol
    u_warm_start_N = np.concatenate([u_sol[1:], u_sol[-1].reshape(1, nu)]) #u_sol
    # CL-MPC
    q_meas, dq_meas = simulator.robot.get_state()
    simulator.robot.pin_robot.framesForwardKinematics(q_meas)
    com = pin.centerOfMass(rmodel, rdata, q_meas)
    hg = np.array(
        simulator.robot.pin_robot.centroidalMomentum(q_meas, qdot_meas)
    )
    x0 =  np.concatenate(
        [
            com, 
            hg,
            q_meas[0:3],
            np.array(
                model.casadi_model.q_minus(
                    x_ref[traj_time_idx+1][12:16] , q_meas[3:7]
                )
            ).squeeze(),
            q_meas[7:],
            qdot_meas
        ]
    )
pybullet.stopStateLogging(logger)